# Powercreep EDA

Original dataset taken from Josh Korngiebel on Kaggle.
https://www.kaggle.com/datasets/jkorn28/pokemon-stats-gens-1-9 

Dataset modified by me in the data prep folder. All operations in this notebook will be using the modified csv file.

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Loading and Previewing Dataset

In [9]:
pokedf = pd.read_csv('pokemon_data_custom.csv')

In [10]:
pokedf.head()

,Dex No,Name,Base Name,Type 1,Type 2,BST,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Generation,Legendary,Mythical,Notable
0,1,Bulbasaur,Bulbasaur,GRASS,POISON,318,45,49,49,65,65,45,1,False,False,False
1,2,Ivysaur,Ivysaur,GRASS,POISON,405,60,62,63,80,80,60,1,False,False,False
2,3,Venusaur,Venusaur,GRASS,POISON,525,80,82,83,100,100,80,1,False,False,False
3,3,Mega Venusaur,Venusaur,GRASS,POISON,625,80,100,123,122,120,80,6,False,False,False
4,4,Charmander,Charmander,FIRE,-,309,39,52,43,60,50,65,1,False,False,False


In [11]:
pokedf.tail()

,Dex No,Name,Base Name,Type 1,Type 2,BST,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Generation,Legendary,Mythical,Notable
1196,1013,Sinistcha,Sinistcha,GRASS,GHOST,508,71,60,106,121,80,70,9,False,False,False
1197,1014,Okidogi,Okidogi,POISON,FIGHT,555,88,128,115,58,86,80,9,True,False,False
1198,1015,Munkidori,Munkidori,POISON,PSYCHIC,555,88,75,66,130,90,106,9,True,False,False
1199,1016,Fezandipiti,Fezandipiti,POISON,FAIRY,555,88,91,82,70,125,99,9,True,False,False
1200,1017,Ogerpon,Ogerpon,GRASS,-,550,80,120,84,60,96,110,9,True,False,False


The dataset contains all currently released Pokemon, complete with 6 base stats for each and their totals. It also contains boolean columns for special types of Pokemon. A more specific choice I made while editing this dataset in the preparation file was to list all Mega Pokemon as the generation that Megas were released in, 6, rather than the Pokemons' home generations.